# Predicció de càncer de pell utilitzant Machine Learning

Llibreries que utilitzarem

In [3]:
import torch
import torch.nn as nn
from torchvision.datasets.folder import ImageFolder
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

Importar les dades que utilitzarem. Al ser imatges fem servir el mètode ImageFolder de Torchvision

- Passem les dades d'imatges a tensors

In [4]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

- Definir el dataset i crear el dataloader

In [5]:
directory = 'data'
bs = 64

train_dataset = datasets.ImageFolder(f"{directory}/train",transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

test_dataset = datasets.ImageFolder(f"{directory}/test", transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

- Farem servir la GPU

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f'Device: {device}')

Device: cuda


- Creació del model neuronal

In [ ]:
class MelanomaDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(),
            nn.MaxPool2d(),
            nn.Conv2d(),
            nn.MaxPool2d(),
            nn.Flatten(),
            nn.Linear(),
            nn.ReLU(),
            nn.Linear()
        )
    
    def forward(self, X):
        X = self.features(X)
        return X